In [327]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=15)


def parses(data):
    return data

data = parses(puzzle.input_data)

In [328]:
from collections import namedtuple

In [349]:
class P(namedtuple('Vec2d', ['x', 'y'])):
    def __add__(self, other):
        return P(self.x+other.x, self.y+other.y)
    def __sub__(self, other):
        return P(self.x-other.x, self.y-other.y)

In [350]:
def as_structs(data):
    board, moves = data.split('\n\n')
    board = [list(row) for row in board.split('\n')]
    moves = list(moves.replace('\n', ''))
    wall, boxes, robot = set(), set(), None
    for x, row in enumerate(board):
        for y, v in enumerate(row):
            if v == '#':
                wall.add(P(x,y))
            if v == 'O' or v == '[':
                boxes.add(P(x,y))
            if v == '@':
                robot = P(x,y)
    dirs = {'<': P(0,-1), '>': P(0,1), 'v': P(1,0), '^': P(-1,0)}
    moves = [dirs[m] for m in moves]
    return wall, boxes, robot, moves

In [351]:
def gps(boxes):
    return sum((100*x+y for x,y in boxes))

In [352]:
def solve_a(data):
    wall, boxes, robot, moves = as_structs(data)
    pos = robot
    for m in moves:
        if pos + m in wall:
            continue
        if pos + m not in boxes:
            pos += m
            continue
        box = pos + m
        while box in boxes:
            box += m
        if box in wall:
            continue
        boxes.add(box)
        boxes.remove(pos+m)
        pos = pos + m

    return gps(boxes)

In [353]:
sample1 = parses("""########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<""")

In [354]:
sample2 = parses("""##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^""")

In [355]:
solve_a(sample2)

10092

In [376]:
solve_b(sample2)

9021

In [356]:
sample3 = parses("""#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^""")

In [357]:
def scale_up(board):
    board = board.replace('#', '##')
    board = board.replace('O', '[]')
    board = board.replace('.', '..')
    board = board.replace('@', '@.')
    return board


In [373]:
def solve_b(data):
    data = scale_up(data)
    wall, boxes, robot, moves = as_structs(data)
    pos = robot
    for m in moves:
        newpos = pos + m
        if newpos in wall:
            # blocked by wall
            continue
        if newpos not in boxes and (newpos - P(0,1)) not in boxes:
            # not blocked by box
            pos = newpos
            continue
            
        # blocked by box
        box = newpos if newpos in boxes else newpos - P(0,1)
        
        would_move = set()
        @cache
        def move(box):
            would_move.add(box)
            needs_free = {
                P(0,1): [box+P(0,2)],
                P(0,-1): [box+m],
                P(1,0): [box+m, box+m+P(0,1)],
                P(-1,0): [box+m, box+m+P(0,1)],
            }[m]

            for required in needs_free:
                if required in wall:
                    return False
                for other_box in (required, required - P(0,1)):
                    if other_box in boxes and not move(other_box):
                        return False
            return True
        
        if not move(box):
            continue
        
        boxes = (boxes - would_move) | {b+m for b in would_move}
        pos = pos + m

    return gps(boxes)

In [374]:
solve_b(data)

1381446

In [153]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=15)

# def parses(data):
#     return data.strip().split('\n')

def scale_up(board):
    board = board.replace('#', '##')
    board = board.replace('O', '[]')
    board = board.replace('.', '..')
    board = board.replace('@', '@.')
    return board

def parses(data):
    board, moves = data.strip().split('\n\n')
#     return board, moves

#     wall, boxes, robot = set(), set(), None
#     for i, row in enumerate(board.split('\n')):
#         for j, v in enumerate(row):
#             z = i + 1j*j
#             if v == '@':
#                 robot = z
#             elif v == '#':
#                 wall.add(z)
#             elif v == 'O':
#                 boxes.add(z)
    
#     tr = {'<': -1j, '>': 1j, 'v': 1, '^': -1}
# #     print(repr(moves))
#     moves = [tr[m] for m in moves.strip('\n').replace('\n','')]
#     return (wall, boxes, robot), moves
    board = scale_up(board)
    
    wall, boxes, robot = set(), set(), None
    for i, row in enumerate(board.split('\n')):
        for j, v in enumerate(row):
            z = i + 1j*j
            if v == '@':
                robot = z
            elif v == '#':
                wall.add(z)
            elif v == '[':
                boxes.add(z)
#             elif v == ']':
#                 boxes.add(z)
    print(i+1,j+1)
    tr = {'<': -1j, '>': 1j, 'v': 1, '^': -1}
#     print(repr(moves))
    moves = [tr[m] for m in moves.strip('\n').replace('\n','')]
    return (wall, boxes, robot), moves
            

# import re
# def parses(data):
#     return [[int(i) for i in re.findall("-?\d+", line)] for line in data.strip().split('\n')]

data = parses(puzzle.input_data)

50 100


In [203]:
rev = {v:k for k, v in {'<': -1j, '>': 1j, 'v': 1, '^': -1}.items()}

In [84]:
print(scale_up("""#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######"""))

##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############


In [88]:
sample1 = parses("""########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<""")

8 16


In [103]:
sample2 = parses("""#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^""")

7 14


In [195]:
def render(wall, boxes, pos):
    N = int(max([i.real for i in wall]))+1
    M = int(max([j.imag for j in wall]))+1
    s = ""
#     print(N,M)
#     print(wall)
    for i in range(N):
        for j in range(M):
            z = i+1j*j
            if pos == z:
                s += '\033[91m@\033[0m'
            elif z in wall:
                s += '#'
            elif z in boxes:
                s += '\033[94m['
            elif z-1j in boxes:
                s += ']\033[0m'
            else:
                s += ' '
        s += '\n'
    print(s)

In [196]:
(wall, boxes, robot), moves = copy.deepcopy(sample2)

In [197]:
render(wall, boxes, robot)

##############
##      ##  ##
##          ##
##    [][]@ ##
##    []    ##
##          ##
##############



In [119]:
from functools import cache

In [132]:

# def verbose(f):
#     def _f(box):
#         b = f(box)
#         print(box,b)
#     return _f

In [146]:
sample3 = parses("""##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^""")

10 20


In [251]:
from functools import cache

In [262]:
def solve_b(data):
    (wall, boxes, robot), moves = copy.deepcopy(data)#     (wall, boxes, robot), moves = copy.deepcopy(sample2)

    pos = robot
    # render(wall, boxes, pos)
    for m in moves:
#         print(f'Move {rev[m]}')
#         render(wall,boxes,pos)
#         print()
#         print(len(boxes))
        newpos = pos + m
        if newpos in wall:
#             print('  wall')
    #         render(wall, boxes, pos)
            continue
        if newpos not in boxes and (newpos-1j) not in boxes:
#             print(f'  Free {m}')

            pos = pos+m
    #         render(wall, boxes, pos)
            continue

        if newpos in boxes:
            box = newpos
        elif newpos-1j in boxes:
            box = newpos-1j
        
        would_move = []
        @cache
        def move(box):
            would_move.append(box)
            if box not in boxes:
                raise ValueError
            if m == -1j:
                if box-1j in wall:
                    return False
                if box-2j not in boxes or move(box-2j):
#                     boxes.remove(box)
#                     boxes.add(box-1j)
                    return True
                return False
            if m == 1j:
                if box+2j in wall:
                    return False
                if box+2j not in boxes or move(box+2j):
#                     boxes.remove(box)
#                     boxes.add(box+1j)
                    return True
                return False
            if m in (1,-1):
                if box+m in wall or box+m+1j in wall:
#                     print(box, False, 1)
                    return False

                infront = [box+d for d in (m, m+1j, m-1j) if box+d in boxes]
                if all(map(move, infront)):
#                     boxes.remove(box)
#                     boxes.add(box+m)
#                     print(box, True)
                    return True

#                 print(box, False)
                return False

#         print(f'Move {rev[m]}')
#         render(wall,boxes,pos)
#         print()

        moves = move(box)
        if moves:
            for box in would_move:
                boxes.remove(box)
            for box in would_move:
                boxes.add(box+m)
#             print('🔥')
            pos = pos + m
#         render(wall,boxes,pos)
#         print(moves)
#         print()
#         print()
        
#         else:
#             print(pos, '❌')

    #     print(f'Move {m}')
    #     render(wall, boxes, pos)

#     render(wall, boxes, pos)
    return gps(boxes)          

    #     while newpos in boxes:
    #         newpos += m
    #     if newpos in wall:
    #         continue
    #     boxes.add(newpos)
    #     pos = pos + m
    #     boxes.remove(pos)

In [263]:
solve_b(sample2)

618

In [264]:
solve_b(sample3)

9021

In [265]:
solve_b(data)

1381446

In [250]:
solve_b(data)

1377424

In [29]:
def gps(boxes):
    total = 0
    for z in boxes:
        x = z.real
        y = z.imag
        total += 100 * x + y
    return int(total)

In [33]:
import copy

In [34]:
def solve_a(data):
    (wall, boxes, robot), moves = copy.deepcopy(data)
    pos = robot
    for m in moves:
        newpos = pos + m
        if newpos in wall:
            continue
        if newpos not in boxes:
            pos = newpos
            continue
        while newpos in boxes:
            newpos += m
        if newpos in wall:
            continue
        boxes.add(newpos)
        pos = pos + m
        boxes.remove(pos)
    #     boxes.remove(newpos)

    return gps(boxes)

In [35]:
sample1

(({(1+0j),
   (1+7j),
   (2+0j),
   (2+1j),
   (2+7j),
   (3+0j),
   (3+7j),
   (4+0j),
   (4+2j),
   (4+7j),
   (5+0j),
   (5+7j),
   (6+0j),
   (6+7j),
   (7+0j),
   (7+1j),
   (7+2j),
   (7+3j),
   (7+4j),
   (7+5j),
   (7+6j),
   (7+7j),
   0j,
   1j,
   2j,
   3j,
   4j,
   5j,
   6j,
   7j},
  {(1+3j), (1+5j), (2+4j), (3+4j), (4+4j), (5+4j)},
  (2+2j)),
 [(-0-1j), -1, -1, 1j, 1j, 1j, 1, 1, (-0-1j), 1, 1j, 1j, 1, (-0-1j), (-0-1j)])

In [36]:
solve_a(sample1)

2028

In [41]:
solve_a(data)

1360570

In [11]:
sample1

(({(1+0j),
   (1+7j),
   (2+0j),
   (2+1j),
   (2+7j),
   (3+0j),
   (3+7j),
   (4+0j),
   (4+2j),
   (4+7j),
   (5+0j),
   (5+7j),
   (6+0j),
   (6+7j),
   (7+0j),
   (7+1j),
   (7+2j),
   (7+3j),
   (7+4j),
   (7+5j),
   (7+6j),
   (7+7j),
   0j,
   1j,
   2j,
   3j,
   4j,
   5j,
   6j,
   7j},
  {(1+3j), (1+5j), (2+4j), (3+4j), (4+4j), (5+4j)},
  (2+2j)),
 [(-0-1j), -1, -1, 1j, 1j, 1j, 1, 1, (-0-1j), 1, 1j, 1j, 1, (-0-1j), (-0-1j)])

In [ ]:
def solve_a(data):
    pass

In [ ]:
solve_a(sample)

In [ ]:
solve_a(data)

In [ ]:
def solve_b(data):
    pass

In [ ]:
solve_b(sample)

In [ ]:
solve_b(data)